# Lab 3b: Colorectal Cancer Histology
*Version 3.0b, Summer Semester 2022*

Based on the learnings of the previous lab and the hands-on examples from the [online-videos](https://www.youtube.com/playlist?list=PL1lqZksc3XHQLC8Yb20IG6bXUQkQStd5n), you will now work on a second convolutional neural network. While the first example was great to see the power of deep learning based on images, we want to apply it to a scenario more directly related to healthcare.

### The Dataset

We'll use the [Colorectal Cancer Histology](https://zenodo.org/record/53169#.XGZemKwzbmG) dataset. It was the basis of an article published to *Nature* in 2016 and is [available for free through Open Access](https://www.nature.com/articles/srep27988). Kather et al. achieved an accuracy of 87.4% for a multiclass scenario. Let's see how far you can get. The dataset was then published under Creative Commons license and very recently added to the [TensorFlow example database](https://www.tensorflow.org/datasets/catalog/colorectal_histology), making it easier for us to load and process the data.

It includes 5,000 RGB histological images, each 150x150px. These have been classified for 8 different targets (labeled as: 'tumor', 'stroma', 'complex', 'lympho', 'debris', 'mucosa', 'adipose', 'empty'). The following image contains representative images of these classes:

![Representative images](https://github.com/andijakl/MachineLearning/raw/main/lab%203%20-%20deep%20learning%20-%20colorectal%20cancer/lab3b-representative-images.jpg) *(a) tumour epithelium, (b) simple stroma, (c) complex stroma (stroma that contains single tumour cells and/or single immune cells), (d) immune cell conglomerates, (e) debris and mucus, (f) mucosal glands, (g) adipose tissue, (h) background.*

The dataset is around 260 MB, which still makes it possible to use a standard laptop without dedicated hardware acceleration for machine learning.

## Your Name

Replace the `raise NotImplementedError` with the code `myname = ""` and assign your name to the variable:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert myname != "", "myname should not be empty"

## 1: Initializing

First, import `numpy` as `np`, `tensorflow` with the alias `tf`. Next, import `pandas` as `pd`; `matplotlib.pyplot` as `plt`. Configure matplotlib to draw images inline.

* If running in the FHSTP JupyterHub or Google Colab environment, all required packages are installed.
* If running a local Anaconda installation, please install the `tensorflow` module (e.g., through the "Environments" UI in Anaconda Navigator).
* If running in Amazon SageMaker, choose the `Python 3 (TensorFlow 2.x Pyton 3.x CPU Optimized)` Kernel.

In [ ]:
# Note: if running on Amazon SageMaker and matplotlib is not installed, uncomment and run the following line:
# !pip install -q matplotlib

In [ ]:
# Place all neccessary imports here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
from packaging import version
assert version.parse(tf.version.VERSION) >= version.parse('2.3.0')

In many real-life cases, you will have a lot of data or an otherwise dynamic process to feed data into your neural network. It doesn't just work to simply load all training examples into memory at once.

TensorFlow contains a module called `tensorflow_datasets` ([tfds](https://www.tensorflow.org/datasets/api_docs/python/tfds)), which can automatically download data and provide it in a suitable format for further processing by TensorFlow. Execute the following code block to import the module:

* If running in the FHSTP JupyterHub or Google Colab environment, all required packages are installed.
* If running a local Anaconda installation, the conda package manager serves `tensorflow_datasets` through the Anaconda Navigator UI, but most likely a very old version (1.2.0). In this case, also install the latest version (4.0.0 or higher) through the pip install code blow.
* If running in Amazon SageMaker, uncomment and run the pip install code below. Note: currently, the SageMaker environment seems to have issues executing this notebook due to wrong versions / missing dependencies.

In [ ]:
# To install or update the tfds version, uncomment and execute the following code line 
# in case you don't have version 4.0.0 or higher installed:
#!pip install -q tensorflow-datasets

In [ ]:
# Pre-defined code: simply execute this cell
import tensorflow_datasets as tfds
assert version.parse(tfds.__version__) >= version.parse('4.0.0')

Next, check which datasets can be loaded through this method. Use the `list_builders()` function of `tfds` (there is also an [online version of this list](https://www.tensorflow.org/datasets/catalog/overview)).

In [ ]:
# List all available datasets from the tensorflow datasets module
# YOUR CODE HERE
raise NotImplementedError()

The example we're looking for is called `colorectal_histology`. The fact that it's contained in `tfds` means that we do not need to worry about downloading the data manually. It also takes care of loading all the individual image files into our input pipeline.

*Note:* the `colorectal_histology_large` dataset is different – it contains 10 high-res 5000x5000px images, each containing more than one tissue types. This would then not just require a classification of the whole tissue sample as we're doing, but additionally a localization of where to find the specific classes.

## 2: Load & Analyze the Dataset

Now, it's time to load the dataset. In this dataset, all 5,000 examples come in one piece – there is no pre-defined train / test / validation split. Therefore, we need to define the split manually.

Use `tfds` to `load` the dataset. Use four parameters:

* First, the dataset name as a string – simply provide the name of the dataset we want to load. Copy and paste the name of our histology dataset from the available datasets we printed before.
* Second, the `split` to use. This dataset provides all samples in a `train` variable. Based on this, we want to use create two sub-dataset variants: 80% for training (`ds_train`), 20% for testing (`ds_test`). Specify this split using `['train[:80%]', 'train[80%:]']` that you assign to the `split` parameter.
* Third, set `as_supervised` to `True`. This automatically creates a dataset that contains the input data and the labels as a python tuple. This means that both are contained in a single returned item.
* To additionally take a look at further information about the dataset, also set `with_info` to `True`

The `tfds.load` function returns two data structures. To assign two structures from return values to variables, simply separate them with a comma:

1. A **tuple** that contains **both datasets**, according to the split. Name them `ds_train` and `ds_test` accordingly. A Python tuple is written like this: `(ds_train, ds_test)`
2. The **info about the dataset** which we requested to load. Store this in a variable `ds_info`.

In [ ]:
# Import the colorectal_histology dataset into the variables: (ds_train, ds_test), ds_info
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert ds_train.element_spec[0].shape == [150, 150, 3]
assert ds_test.element_spec[0].shape == [150, 150, 3]
assert isinstance(ds_train, tf.data.Dataset)
assert type(ds_info) == tfds.core.dataset_info.DatasetInfo
assert ds_info.name == 'colorectal_histology'

Next, print the contents of the `ds_info` variable to get some basic info about the dataset.

In [ ]:
# Print the dataset info
# YOUR CODE HERE
raise NotImplementedError()

The `featues["label"]` of `ds_info` contains a bit more information about the target labels we want to classify for. First, assign its `num_classes` property to a variable called `class_count` and also print this new variable you just created:

In [ ]:
# Assign num_classes contained in ds_info to a variable class_count and print it
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert class_count > 0

You can also retrieve the short target names for each class. To get more information about these, refer to the [original paper](https://www.nature.com/articles/srep27988). Assign the property `names` of `features["label"]` from your dataset info (`ds_info`) to a new variable `class_names` and also print it:

In [ ]:
# Assign the class names to a variable class_names and print it
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert class_names[0] == 'tumor'

## 3: Take a look at examples

Now that we loaded and split the dataset, let's take a look at an instance.

In our previous examples, we usually had all data in memory at once (for example, as a Numpy array). Especially for larger image datasets that can easily reach many gigabytes, this is often not possible – the memory requirements would just be too high. Our `ds_train` variable is actually an instance of the [DataSet class](https://www.tensorflow.org/api_docs/python/tf/data/Dataset), which can represent potentially large datasets and provides ways to dynamically load just the parts of the dataset into memory that are currently needed.

A simple way to get one example out of the dataset is to use the function `take(1)` from `ds_train`. Store this in a new variable, for example `ds_train_example`. This creates a representation of the dataset with just the first element loaded.

In [ ]:
# Store the first data item into a variable ds_train_example
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert len(ds_train_example) == 1

Next, just try to print the `ds_train_example` variable to see what types it contains:

In [ ]:
# Print the dataset example
# YOUR CODE HERE
raise NotImplementedError()

You should see that `ds_train_example` contains two items:

1. The first is a tissue image. It has dimensions of `150, 150, 3`. This means that we have `150x150px` as the image size, with `3` color channels. The data type is `uint8`, which is an unsigned integer with 8 bits – so it can store values from `0..255`, which is exactly what we need for colors.
2. The second is of type `int64` with just a single element. This contains the label.

So, both the data and the label are stored in a single datastructure.

## 4: Draw the example image

Even though we only loaded a single item with `take()`, it is still contained in a data structure that could potentially store more than one item. Therefore, we need to iterate over `ds_train_example` to access the items. The easiest way is a `for` loop. In Python, it can directly extract both contained data items of the tuple into separate variables.

The first line of your code should then look like this: `for img, label in ds_train_example:`

Inside the `for` loop, execute two things:

1. **Print the *shape* of the `img` variable, as well as the `label`.** Use Python string formatting that we took a look at the beginning of the course to create a nicely readable output.  
Note: if you directly print the `label` variable, you will see that it is a [Tensor](https://www.tensorflow.org/guide/tensor). This is a special data structure of TensorFlow, but very similar to a numpy array. To print the label as text, convert it to a numpy data type through `label.numpy()`.
2. **Plot the image.** At the beginning, you imported `matplotlib.pylot` with an alias. It contains a convenient function called `imshow()`. Use this to plot the image `img` to the screen. This is just a single and very short line of code.

Remember that in Python, everything that belongs to a `for` loop needs to have the same indentation level.

In [ ]:
# Use a for loop to iterate over ds_train_example.
# Inside the loop, print the shape and label of the current sample, and use plt.imshow to view the image.
# YOUR CODE HERE
raise NotImplementedError()

In the following cell, assign the textual name of the image label to a new variable called `example_label`. Look up the label number with the list of label names we printed before (or even if you like the full name in the [research paper](https://www.nature.com/articles/srep27988)).

In [ ]:
# Create a (string) variable called example_label where you assign the textuaL label name of the sample you printed above.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert example_label != None
import hashlib
assert hashlib.blake2b(example_label.casefold().encode()).hexdigest() == '601d93b5ed6586571134fb5efb8e8c0ea8a4b49e9ab560a3315e21c0237e8843c9bc1ffe4153f61ad4212770276c9c0382d9ed1a0965b4eb7c2de31969189615'

The method you created above always works for datasets and can print information about a contained item. The *Tensorflow Datasets* module we're using in this example actually has an even nicer method of showing the first few examples of a dataset.

The corresponding function is `tfds.show_examples`. It needs two parameters: first the dataset itself (e.g,. `ds_train` or `ds_test`), and second the `ds_info` containing the label names.

Run the function to see the 9 first examples of the `ds_test` dataset. Assign the returned object to a new variable called `fig` to avoid showing it two times.

In [ ]:
# Use tfds.show_examples to print the first examples of the ds_test dataset
# YOUR CODE HERE
raise NotImplementedError()

One more example so that you can see that it's all connected :)

You can also directly convert the data to a Pandas dataframe. Use `tfds.as_dataframe`. For the first parameter, specify some samples you want to visualize - e.g., use the `take()`  function as before to take the first 4 examples from `ds_train`. As the second parameter, specify `ds_info`.

In [ ]:
# Use tfds.as_dataframe to convert some samples to a Pandas Dataframe and use this to visualize the examples.
# Take the first 4 samples of ds_train, and also provide ds_info as the second parameter.
# YOUR CODE HERE
raise NotImplementedError()

## 5: Image Pre-Processing

First of all, let's take a look at the raw data of an image. Simply print the `img` variable:

In [ ]:
# Print the img variable
# YOUR CODE HERE
raise NotImplementedError()

You can see in the output that `img` is a multi-dimensional array. The image contains the intensity values of all three colors (red, green, blue). These are scaled from 0..255. As we learned before, Neural Networks work best if you normalize the images. You should scale the values to a range 0..1.

## 6: Input Pipeline

Execute the following code block so that Python knows about our normalization function. It's taken from the [TensorFlow documentation](https://www.tensorflow.org/datasets/performances#caching_the_dataset). But essentially, it's the same as in our [MNIST example from the YouTube video](https://youtu.be/yNlBNp8KORA?t=777); the operation simply isn't performed on the data in memory (which is impossible for huge datasets), but instead added to the input pipeline.

In [ ]:
# Pre-defined code: simply execute this cell
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

We just defined a function which Tensorflow can apply to the images from our dataset. Next, we need to define the pipeline of how the actual raw images get to the machine learning model.

The first step is to apply the normalization function to the image data. This is done through the `map` function of the dataset. Finally, simply store the updated pipeline in the original variable. The line should therefore look like this: `ds_train = ds_train.map(normalize_img)`

In [ ]:
# Apply the normalization function to the image data
# YOUR CODE HERE
raise NotImplementedError()

As the next step in our input pipeline, let's shuffle the data. We will go through the dataset multiple times (called *epochs*). It's a good idea to have the order of items randomized each time – especially, as we don't know if the classes are occuring randomly in the dataset. Otherwise, it could happen that a particular batch of the training data only contains examples of a single class, which makes it difficult for the model to learn how to distinguish classes.

In the same way as before, add a `shuffle(4000)` step to our `ds_train` pipeline. `4000` is just the number of data items in our training set (80% of the 5000), so that the whole dataset is shuffled. Remember to re-assign the returned value to `ds_train` again, like before.

In [ ]:
# Add shuffle with a parameter 4000 to ds_train and assign it back to ds_train 
# (like in the previous step)
# YOUR CODE HERE
raise NotImplementedError()

As we have a large amount of data, it's a good idea to use *batching* with mini-batches as introduced in the theoretical part. This means that the model can update the neural network weights after each batch, and doesn't need to run through the entire dataset before it can update the weights. This makes learning faster and usually helps a bit to prevent over-fitting. As we have 8 different classes, a bigger batch size of for example 128 is good, so that multiple classes are present in each batch.

Like before, add `batch(128)` to our input pipeline:

In [ ]:
# Add batching to the training dataset, with a batch size of 128
# YOUR CODE HERE
raise NotImplementedError()

The last two steps are optional, but help a bit to make machine learing faster. They make sure that TensorFlow optimally uses the different cores of your CPU and GPU so that for example one processor core of your PC can prepare the next batch, while another core is busy training based on another batch. Simply execute the next code block to add these steps to our pre-processing pipeline:

In [ ]:
# Pre-defined code: simply execute this cell
ds_train = ds_train.cache()
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

Great – our training data pipeline is ready! Now, we need to perform the same steps on the `ds_test` data. Use the same code as before in the following cell, but make sure you always replace `ds_train` with `ds_test`. The only difference: do not use the `shuffle()` step. Shuffling the test data is not needed to evaluate the performance of our machine learning model.

In [ ]:
# Create a pipeline for the ds_test dataset. Include the following steps:
# 1. map using the normalize_img function
# 2. batch(128)
# 3. cache
# 4. prefetch
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
# If the tests fail, restart and run all cells. You might have executed
# code blocks that add steps to the input pipeline multiple times, which
# can have unintended side-effects.
ds_example = ds_test.take(1)
for (img,x) in ds_example:
    assert img.shape == (128,150,150,3)
    assert (img[0][0][0][0] >= 0)
    assert (img[0][0][0][0] <= 1)
ds_example = ds_train.take(1)
for (img,x) in ds_example:
    assert img.shape == (128,150,150,3)
    assert (img[0][0][0][0] >= 0)
    assert (img[0][0][0][0] <= 1)

## 7: Build the Convolutional Neural Network

Now, we're finally at the step where we define the structure of the neural network! The following would be a good architecture to start with:

![Sample architecture of the CNN](https://github.com/andijakl/MachineLearning/raw/main/lab%203%20-%20deep%20learning%20-%20colorectal%20cancer/lab3b-architecture.png)*Sample architecture of the CNN*

Usually, a CNN has multiple convolutional layers, each followed by a max-pooling layer. At the end, after a flattening layer, two dense (fully connected) layers ensure that the outputs of the convolutional blocks are classified. The last layer is using the *softmax* activation function with 8 neurons, so that the probability for each of the 8 classes can be predicted.

**Your task:** build an architecture like the one in the image above. You can use the architecture implementation of Lab 3a as reference.

**Some hints:**

* Typical definition of a convolutional layer:  
`tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))`  
Indiviual parameters:  
  1. Define the **number of filters**. You could for example start with 32, and then go up to 64 layers for the following layers. Refer to the image above. The number of layers is printed above the image, e.g., `32@148x148` means that we have 3 layers and the images have a shape of `148x148` (missing 1 border pixel on each side due to the `3x3` convolution).
  2. **Size of the filters** in this layer, for example `3x3`. Note that the filter size is not visible in the image above; the size is indicating the image size at this stage - it starts at `150x150`, gets a bit smaller with each convolutional layer due to the border pixels and is halved with each max-pool layer.
  3. **Activation function** to use. Use the `relu` function.
  4. The **input shape**: only necessary for the first layer. In this case, it's the image size that we already took a look at before – 150x150px with 3 colors.
* Typical definition of a Max-Pool layer:  
`tf.keras.layers.MaxPooling2D((2, 2))`  
There is only a single parameter – the size of the pool. Always use a `2x2` pool. This already halves both the width and height of the data, resulting in a 1/4 of the original size. As we start with rather small 150x150px images, you shouldn't go too low too quickly.
* The flattening is simple and doesn't have parameters we need to set:  
`tf.keras.layers.Flatten()`
* The last two layers are dense layers, like:  
`tf.keras.layers.Dense(8, activation='softmax')`  
Use 128 nodes for the first dense layer and the `relu` activation function. The second dense layer should have 8 nodes (according to the target classes) with the `softmax` activation function.

In [ ]:
# Create a variable "model" and use the Sequential() function to add a list of layers according to the
# description and the image above.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert type(model) == tf.keras.models.Sequential
assert type(model.layers[0]) == tf.keras.layers.Conv2D
assert type(model.layers[1]) == tf.keras.layers.MaxPooling2D 
assert type(model.layers[2]) == tf.keras.layers.Conv2D
assert type(model.layers[3]) == tf.keras.layers.MaxPooling2D 
assert type(model.layers[4]) == tf.keras.layers.Conv2D
assert type(model.layers[5]) == tf.keras.layers.MaxPooling2D 
assert type(model.layers[6]) == tf.keras.layers.Flatten 
assert type(model.layers[7]) == tf.keras.layers.Dense 
assert type(model.layers[8]) == tf.keras.layers.Dense 
assert model.layers[0].output_shape == (None, 148, 148, 32)
assert model.layers[1].output_shape == (None, 74, 74, 32)
assert model.layers[2].output_shape == (None, 72, 72, 64)
assert model.layers[3].output_shape == (None, 36, 36, 64)
assert model.layers[4].output_shape == (None, 34, 34, 64)
assert model.layers[5].output_shape == (None, 17, 17, 64)
assert model.layers[6].output_shape == (None, 18496)
assert model.layers[7].output_shape == (None, 128)
assert model.layers[8].output_shape == (None, 8)

## 8: Compile the Model

After the model is defined, you also need to compile it. Use:

* **Loss function:** `tf.keras.losses.SparceCategoricalCrossentropy()`. You do *not* need to set `from_logits=True` like in the previous example. Our last layer is `softmax`, so already results in a probability-like distribution that can be directly used by the categorical cross entropy loss.
* **Optimizer:** use `tf.keras.optimizers.Adam()`, like in the previous example.
* **Metrics:** use `accuracy`.

In [ ]:
# Compile the model according to the settings described above
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert type(model.optimizer) == tf.keras.optimizers.Adam
assert type(model.loss) == tf.keras.losses.SparseCategoricalCrossentropy

Next, print the model summary. Note the total number of *trainable parameters*.

In [ ]:
# Print the model summary
# YOUR CODE HERE
raise NotImplementedError()

How many parameters will need to be trained by our neural network? Create a variable `train_parameters` and store the number as an integer (do not use thousand separators).

In [ ]:
# Create a variable train_parameters and insert the value of trainable params 
# from the model summary above
train_parameters = 0
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute.
assert train_parameters > 0
assert isinstance(train_parameters, int)

For reference, if you built the model exactly as shown in the architecture image, the summary should look like this:

<div>
<img src="https://github.com/andijakl/MachineLearning/raw/main/lab%203%20-%20deep%20learning%20-%20colorectal%20cancer/lab3b-model-summary.png" alt="Example model summary" width="300"/>
</div>

## 9: Training

Prepare a cup of coffee / tea for the next step. Depending on your computer speed, this might take several minutes. After all, we're working with real data and are training a deep neural network.

Use the `fit` function of the model to train it based on `ds_train`. Also use the parameter `epochs` and train for at least 2 epochs, depending on your computer speed. This specifies how many iterations the training should make through the whole dataset (note that weight updates are performed already after each batch of 128 examples).

With the CNN architecture from above, you can still get better results if you train for more epochs – but of course, it'll take a longer time. So let's start low; you can always increase the number of epochs once you know that you're on the right path and everything works.

Save the returned training history in a variable called `history`, so that we can then visualize how loss and accuracy improved with each epoch.

In [ ]:
# Fit the model based on the training data for 10 epochs.
# Save the returned training info in a variable called history.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert history.history['loss'][0] > 0
assert history.history['accuracy'][0] > 0
assert history.history['loss'][1] < history.history['loss'][0]
assert history.history['accuracy'][1] > history.history['accuracy'][0]

## 10: Visualize the History

Now that your model is trained, let's visualize accuracy and loss. Like in the example from the lecture, Pandas provides an easy way to visualize the training history. Look up the code from [the YouTube video](https://youtu.be/yNlBNp8KORA?t=2462) and use it to draw the diagram:

In [ ]:
# Convert history.history into a pandas DataFrame and use its plot function
# to visualize how the accuracy increased during training and the loss was decreased.
# If focusing on the accuracy (which can only be between 0..1), it makes sense to
# limit the y axis to a range from (0,1).
# YOUR CODE HERE
raise NotImplementedError()

## 11: Evaluate the Model

As you know, the accuracy on the training data is useful. But even more important is the performance of your neural network on new, unseen data.

For this reason, we split the 5,000 examples into `ds_train` and `ds_test` at the beginning. Use the model's `evaluate()` function to compute the accuracy it achieves on the `ds_test` dataset! Most likely, it'll be lower than on the training data. But it should still be a really good result. Remember that we're distinguishing between 8 different target classes!

In [ ]:
# Evaluate the model with the test set.
# Save the results in a variable eval_results. It will contain the loss and accuracy values
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert eval_results[0] > 0
assert eval_results[1] > 0.5
assert eval_results[1] < 0.9

## 12: Predict Examples

Let's see how the model performed on some examples of the test set.

First, let's take the first piece of our `ds_test` dataset. We're using the same method as before with `take(1)`. Store the returned data in a new variable called `ds_predict_example`.

As `ds_test` is now accessed through the TensorFlow input pipeline, we don't just get 1 example, but instead 1 batch – consisting of 128 examples (our batch size). 

In [ ]:
# Take one batch from ds_test and store it in a variable called ds_predict_example
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert list(ds_predict_example.as_numpy_iterator())[0][1].size == 128

Now, let's use our trained model to predict the probabilities from our dataset slice. Call the `predict()` function of our model. Supply the new variable `ds_predict_example` that we just created as argument. Store the outputs in a new variable called `predictions`.

In [ ]:
# Predict probabilities for all 128 examples.
# Store prediction results in variable predictions
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert predictions.shape == (128,8)

Let's take a look at the raw probabilities. For visualizing tabular data, Pandas is always a good choice, as it provides a nicer formatting. Create a Pandas dataframe called `df_predictions`. Give it our `predictions` variable as input.

To see the class labels as column labels, assign these to the `columns` argument when creating the dataframe. As we already printed at the beginning of this notebook, you can access an array of label names through `ds_info.features["label"].names`.

Note: You don't see an output of this line.

In [ ]:
# Create a dataframe based on the predictions variable.
# Call the dataframe df_predictions
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert type(df_predictions) == pd.DataFrame
assert df_predictions.size == 1024
assert df_predictions.shape == (128, 8)

Next, let's show the dataframe – simply write the `df_predictions` variable name into the next cell. You'll see that it outputs the full numbers, which are not very readable. Therefore, modify the line and call the function `.round(decimals=2)` on the dataframe.

The columns correspond to the classes with their respective labels. The rows to the data items (`0..127`, the batch size).

In [ ]:
# Display uses the nice way of drawing Pandas Dataframes, even if there are multiple outputs in this cell
# YOUR CODE HERE
raise NotImplementedError()

You'll see that the neural network usually wasn't 100% sure what is the best class. Sometimes, the maximum in each row is more distinct, in other cases the network has a harder time to decide between classes.

## 13: Visualize Predictions

Let's take a look at individual data items, print their true label, the predicted label and the image itself.

In the code block below, the variables have already been defined. Append your code to fill in these variables. The following code block then visualizes the variables, with short descriptions.

**Your Task:** 
The variable called `show_item` selects which item in our batch to show. Set it to a number between `0` and `127`. This is the item from the predicted batch that we want to visualize.

Next, create the **loop** -> `for img, label in ds_predict_example:` (as before) to get data out of our dataset. As said, the dataset is organized in batches, consisting of 128 examples. Our `for`-loop doesn't return an individual item, but the whole batch. (Even though we only retrieved a single batch. Thus, the loop is only executed once). However, we'll just visualize a single item from the batch, with the index that we've stored in `show_item`. 

**Inside the loop**, assign the info of the single item into the four pre-defined variables:

1. **The true label (use variable: `true_label` for the final result):** the `label` variable from the for loop contains all the 128 labels of the batch. Therefore, simply access the item we need from the array: `label[show_item]`. As before, this returns a tensor. Make it easily printable by converting it to a numpy item with `.numpy()`. We also want to get the label instead of the numberic class. You can get this through `ds_info.features["label"].names[<numeric class label>]`
2. **The predicted label (variable: `predicted_label`):** our neural network always predicts the class that has the max probability. Our predictions are stored in the Pandas dataframe `df_predictions`, which we just created in the previous step. There's an easy way to get what we want – just apply both statements on the dataframe and store the final result in `predicted_label`:
  1. **Accessing rows:** to access a row of a Pandas dataframe by its index, use `df_predictions.iloc[show_item]`. This returns all class probabilities for a single item.
  2. **Label of the maximum value:** the `.max()` function gives you the maximum value (= probability) in this row. While useful, here we would actually need the class label (= column index) with the max probability instead. The `.idxmax()` function does just that, so use it on the row you just retrieved.
3. **All probabilities (variable: `predicted_label_probs`):** To get an impression of how sure the network was about its prediction, also retrieve all probabilities. In the same way as before, access a single row of `df_predictions` using `iloc`. Then, `round()` the output to *two* decimals.
4. **Show the image (variable: `img_data`):** To print the image later, we'll convert the raw data of the image to a numpy array. To do this, call `.numpy()` on `img[show_item]`.

In [ ]:
# Do not modify the variables created here. They will be assigend to later.
true_label = None
predicted_label = None
predicted_label_probs = None
img_data = None

# Modify the following line to choose which item to visualize.
# You can assign any number to show_item from 0..127
show_item = None

# Start coding below, according to the instructions from the block above to fill the variables.
# YOUR CODE HERE
raise NotImplementedError()

After coding the loop above, execute the following block of code to see the output:

In [ ]:
# No need to modify this code, just execute it after writing your code
# in the previous code block. It should visualize the results of the prediction of a single item.
print("Showing item:", show_item)
print("True label:", true_label)
print("Predicted label with max probability:", predicted_label)
print("Predicted label probabilities:")
print(predicted_label_probs)
plt.imshow(img_data)

For reference, this is an example of what your output could look like. Note that the probabilities are most likely different, depending on how long you trained the model. Also the item could be a different one.

<div>
<img src="https://github.com/andijakl/MachineLearning/raw/main/lab%203%20-%20deep%20learning%20-%20colorectal%20cancer/lab3b-prediction.png" alt="Example prediction" width="250"/>
</div>

In [ ]:
# Pre-defined tests to check your code. Do not modify, just execute this cell.
assert show_item >= 0 and show_item < 128
assert true_label is not None
assert predicted_label is not None
assert predicted_label_probs.size == 8
assert np.isclose(np.sum(predicted_label_probs), 1.0, rtol=0.1), "Should be close to 0.99 / 1.00"
assert img_data.shape == (150, 150, 3)

## Next Steps (Optional)

When you've reached this point, it's time to hand in your exercise!

But of course, now the fun part could start - trying to figure out ways how to improve the model to achieve better results. You could add additional convolutional & max-pool layers to your model. Or of course, just train for more epochs – even though you might run into overfitting if you train too long, which would then be visible if your training accuracy is far above the test accuracy. You could tweak the number or size of the filters.

But that's outside of the exercise. Make a private copy of the notebook if you wish to modify the network structure or parameters!

*(Note: with this approach, you're optimizing the results for the `ds_test` set, to make its accuracy as high as possible during evaluation. Thus, you're probably overfitting on the test set. To circumvent that, a usual approach is to have training data for learning the model, validation data to improve the model parameters, and use the test set only to get a good real-life estimate of the final model. We didn't go this route in this example, so that we have more examples for the training & testing sets. The main goal of this lab is to get the neural network to work. But if you enjoyed this work so far, there's a whole world of great ways of how you can fine-tune the model, and strategies for preventing overfitting (e.g., dropout) that you can look into. But these are all details – in this lecture, you've come a long way and can be really proud of what you achieved!)*